In [1]:
import os
import json
import pandas as pd
import traceback
from typing import Dict, Any

In [2]:
from langchain_openai import ChatOpenAI

In [3]:
from dotenv import load_dotenv

load_dotenv()  # reads variables from a .env file and sets them in os.environ

True

In [4]:
KEY = os.getenv("OPENAI_API_KEY")

In [5]:
llm = ChatOpenAI(model="gpt-4.1-nano",openai_api_key = KEY ,temperature=0.5)

In [41]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_core.output_parsers import JsonOutputParser, StrOutputParser
from langchain_community.callbacks import get_openai_callback
import PyPDF2

In [7]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [8]:
TEMPLATE = """
Text: {text}
You are an expert MCQ maker. Given the above text, it is your job to
create a quiz of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like RESPONSE_JSON below and use it as a guide.
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}
"""

In [10]:
quiz_generation_prompt = ChatPromptTemplate.from_template(
    TEMPLATE + "\n{format_instructions}"
)


In [11]:
quiz_parser = JsonOutputParser()

In [13]:
quiz_chain = quiz_generation_prompt | llm | quiz_parser

In [15]:
TEMPLATE2 = """
You are an expert English grammarian and writer. Given a Multiple Choice Quiz for {subject} students,
you need to evaluate the complexity of the questions and give a complete analysis of the quiz. 
Only use at max 50 words for complexity analysis. 
If the quiz is not at par with the cognitive and analytical abilities of the students,
update the quiz questions which need to be changed and change the tone such that it perfectly fits the student abilities.

Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [16]:
quiz_evaluation_prompt = ChatPromptTemplate.from_template(TEMPLATE2)

In [17]:
review_chain = quiz_evaluation_prompt | llm | StrOutputParser()

In [20]:
generate_evaluate_chain = (
    RunnablePassthrough.assign(
        quiz=quiz_chain   # quiz_chain uses JsonOutputParser
    )
    | RunnableParallel(
        quiz=lambda x: x["quiz"],     # pass quiz forward
        review=review_chain           # review_chain uses StrOutputParser
    )
)

In [22]:
print(type(generate_evaluate_chain))

<class 'langchain_core.runnables.base.RunnableSequence'>


In [23]:
file_path = r"E:\Projects\End_to_End_MCQ_Generattor\data.txt"

In [24]:
with open(file_path, 'r') as file:
    TEXT = file.read()

In [25]:
print(TEXT)

"AI" redirects here. For other uses, see AI (disambiguation) and Artificial intelligence (disambiguation).
Part of a series on
Artificial intelligence (AI)

Major goals
Approaches
Applications
Philosophy
History
Controversies
Glossary
vte
Artificial intelligence (AI) is the capability of computational systems to perform tasks typically associated with human intelligence, such as learning, reasoning, problem-solving, perception, and decision-making. It is a field of research in computer science that develops and studies methods and software that enable machines to perceive their environment and use learning and intelligence to take actions that maximize their chances of achieving defined goals.[1]

High-profile applications of AI include advanced web search engines (e.g., Google Search); recommendation systems (used by YouTube, Amazon, and Netflix); virtual assistants (e.g., Google Assistant, Siri, and Alexa); autonomous vehicles (e.g., Waymo); generative and creative tools (e.g., langu

In [26]:
# Serialize the Python dictionary into a JSON-formatted string
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [27]:
NUMBER=5 
SUBJECT="Artificial intelligence"
TONE="simple"

In [28]:
print(type(generate_evaluate_chain))


<class 'langchain_core.runnables.base.RunnableSequence'>


In [29]:
#https://python.langchain.com/docs/modules/model_io/llms/token_usage_tracking

#How to setup Token Usage Tracking in LangChain
with get_openai_callback() as cb:
    response = generate_evaluate_chain.invoke({
        "text": TEXT,
        "number": NUMBER,
        "subject": SUBJECT,
        "tone": TONE,
        "response_json": RESPONSE_JSON,
        "format_instructions": quiz_parser.get_format_instructions()
    })

In [30]:
print(f"Total Tokens: {cb.total_tokens}")
print(f"Prompt Tokens: {cb.prompt_tokens}")
print(f"Completion Tokens: {cb.completion_tokens}")
print(f"Total Cost (USD): ${cb.total_cost:.6f}")

Total Tokens: 3828
Prompt Tokens: 3040
Completion Tokens: 788
Total Cost (USD): $0.000475


In [32]:
response

{'quiz': {'1': {'mcq': 'What is artificial intelligence (AI) primarily about?',
   'options': {'a': 'Creating physical robots',
    'b': 'Enabling machines to perform tasks that usually require human intelligence',
    'c': 'Designing computer hardware',
    'd': 'Programming simple calculators'},
   'correct': 'b'},
  '2': {'mcq': 'Which of the following is an example of a high-profile AI application?',
   'options': {'a': 'Word processors',
    'b': 'Google Search',
    'c': 'Typewriters',
    'd': 'Simple calculators'},
   'correct': 'b'},
  '3': {'mcq': 'What does machine learning involve?',
   'options': {'a': 'Machines learning to build hardware',
    'b': 'Programs improving their performance automatically through data',
    'c': 'Humans teaching robots manually',
    'd': 'Machines only following fixed instructions'},
   'correct': 'b'},
  '4': {'mcq': 'What is natural language processing (NLP) used for?',
   'options': {'a': 'Processing visual images',
    'b': 'Understanding 

In [33]:
quiz=response.get("quiz")

In [35]:
quiz = response["quiz"]

In [36]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [37]:
quiz_table_data

[{'MCQ': 'What is artificial intelligence (AI) primarily about?',
  'Choices': 'a: Creating physical robots | b: Enabling machines to perform tasks that usually require human intelligence | c: Designing computer hardware | d: Programming simple calculators',
  'Correct': 'b'},
 {'MCQ': 'Which of the following is an example of a high-profile AI application?',
  'Choices': 'a: Word processors | b: Google Search | c: Typewriters | d: Simple calculators',
  'Correct': 'b'},
 {'MCQ': 'What does machine learning involve?',
  'Choices': 'a: Machines learning to build hardware | b: Programs improving their performance automatically through data | c: Humans teaching robots manually | d: Machines only following fixed instructions',
  'Correct': 'b'},
 {'MCQ': 'What is natural language processing (NLP) used for?',
  'Choices': 'a: Processing visual images | b: Understanding and generating human languages | c: Controlling robotic arms | d: Playing chess',
  'Correct': 'b'},
 {'MCQ': 'What would a 

In [38]:
quiz=pd.DataFrame(quiz_table_data)

In [39]:
quiz.to_csv("artificial_intelligence.csv",index=False)

In [40]:
from datetime import datetime
datetime.now().strftime('%m_%d_%Y_%H_%M_%S')

'02_01_2026_23_52_43'